In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [16]:
import pandas as pd
import numpy as np
from posts.functions import *

In [3]:
posts = pd.read_excel('500_posts.xlsx') # the only way I found to get up to 500 post_ids for specific time range is through web interface

In [4]:
gen_df = pd.DataFrame()
met_df = pd.DataFrame()

In [20]:
for post_id in posts.post_id.tolist()[:100]:
    gen_data = get_post_general_data(post_id)
    met_data = get_post_metrics_data(post_id)

    gen_df = gen_df.append(gen_data, ignore_index=True)
    met_df = met_df.append(met_data, ignore_index=True)

In [21]:
total_df = gen_df.merge(met_df)

In [22]:
total_df.to_feather('med_500_posts_data')

In [23]:
# now let's clean it a bit

In [24]:
total_df['created_time'] = pd.to_datetime(total_df['created_time'])
total_df['extracted_time'] = pd.to_datetime(total_df['extracted_time'])

In [25]:
total_df.dtypes

post_id                                        object
created_time                      datetime64[ns, UTC]
message                                        object
media_type                                     object
original_url                                   object
post_url                                       object
extracted_time                    datetime64[ns, UTC]
post_impressions                              float64
post_impressions_paid                         float64
post_clicks_other                             float64
post_clicks_link                              float64
post_reactions_like                           float64
post_reactions_love                           float64
post_reactions_haha                           float64
post_reactions_sorry                          float64
post_reactions_wow                            float64
post_reactions_anger                          float64
post_neg_fb_hide_all                          float64
post_neg_fb_hide_clicks     

In [26]:
object_columns = [col for col in total_df.columns if total_df[col].dtype.kind in 'O'] # it's capital o - stands for Object
# https://numpy.org/doc/stable/reference/generated/numpy.dtype.kind.html
total_df[object_columns] = total_df[object_columns].apply(pd.to_numeric, errors='ignore')

total_df.to_feather('med_500_posts_data')
# check data - follow the fb link and compare numbers with its interface
# total_df.sample(5)
# everything is correct!

In [27]:
#data overview
print(f"Posts were created from {total_df['created_time'].min()} to {total_df['created_time'].max()}.")
print(f"The most early extraction time is {total_df['extracted_time'].min()}.")
print(f"So, all posts' metrics are collected at least after {round((total_df['extracted_time'].min()- total_df['created_time'].max())/np.timedelta64(1, 'h'))} hours.")

Posts were created from 2022-05-09 16:52:10+00:00 to 2022-05-11 10:30:14+00:00.
The most early extraction time is 2022-05-23 10:04:01+00:00.
So, all posts' metrics are collected at least after 288 hours.


In [28]:
# posts per day
daily_posts = total_df.loc[:,['created_time', 'post_id']]
daily_posts['dt'] = daily_posts['created_time'].dt.date

In [29]:
daily_posts.groupby('dt')['post_id'].count()

dt
2022-05-09     14
2022-05-10     64
2022-05-11    102
Name: post_id, dtype: int64

In [30]:
# calculate rates
click_rates_df = total_df[total_df['media_type']=='link'].loc[:, ['post_id', 'original_url','post_url', 'post_impressions', 'post_clicks_link']]

In [31]:
click_rates_df['click_rate'] = round(100*click_rates_df['post_clicks_link']/click_rates_df['post_impressions'], 4)

In [32]:
click_rates_df['click_rate_rank'] = click_rates_df['click_rate'].rank(method='max')
click_rates_df['impressions_rank'] = click_rates_df['post_impressions'].rank(method='max')
click_rates_df['post_clicks_link_rank'] = click_rates_df['post_clicks_link'].rank(method='max')

In [33]:
antitop_20 = click_rates_df[(click_rates_df['click_rate_rank']<=20)|(click_rates_df['impressions_rank']<=20)|(click_rates_df['post_clicks_link_rank']<=20)]

In [ ]:
antitop_20.to_excel('FB_anti_top.xlsx')